# Search Algorithms

In this project, the aim is to solve a predefined problem using various search algorithms and compare their qualities. BFS, IDS and A* are the algorithms used for this problem.

## Problem Description

A player starts at location (0,0) of a $n\times m$ map. some cells are blocked by obstacles. at each steps, the player can choose to move up, down, left or right if its path is not blocked by walls or obstacles. each cell may contain one of the following elements:

  - <b>Potion:</b> When player enters a cell with potion, he collects the potion.
    
    
  - <b>Pill:</b> When player enters a cell with pill, he consumes the pill and a new player appears at location (n-1,0). afterwards, only one of the players may move at each step 
    
The goal is to gather all players at location (n-1,m-1) and collect all of the potions in the map.

## Modeling

At first, the problem should be modeled as a graph so that the search algorithms can be applied to it in future steps. Each node representes one ditinct state of the map. Bellow, the components of the model is explained:

 - <b>Initial State:</b> The initial state of the map is given as input in this problem, and the player always starts at bottom left corner of the map.
 
 
 - <b>Actions:</b> At each step,one of the players can move either up, down, left or right. However, sometimes the players may be prevented from taking certain actions due to walls and obstacles.
 
 
 - <b>Transition Model:</b> A state is recognized by this properties: 1- location of each player, 2- remaining potions, 3-location of each component(potion-pill-obstacle..) our transition model takes a state and an action as input and determines the next state properties using the rules described in problem description. if an action is impossible to take function returns 0 as output. 
 
 
 - <b>Goal State:</b> The goal state is when all of the players have reached the top right corner of the map and collected all potions in the map.
 
 
 - <b>Path Cost:</b> In this problem, the cost of all actions are equal
 

### Model Implementation

In [259]:
import time
import copy as cp
import heapq as hp
PLAYER='P'
POTION='M'
PILL='D'
OBSTACLE='O'

- <b>States:</b> we use dictionary to store the properties of each state. we choose dictionary because of its simplicity and built-in functions

<b>note:</b> Even though in was possible to obtain the number of potions by location of each component, we choose to store it separately to simplify some functions and avoid possible time consuming calculations.

 - <b>Initiat State:</b>

In [2]:
def read_initial_state(file_name):
    file = open(file_name)
    row,col=file.readline().split()
    potion_count,pill_count=file.readline().split()
    initial_map={"row": int(row) , "col": int(col) , "potion_count": int(potion_count), "ploc":[]}
    for _ in range(int(potion_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=POTION
    for _ in range(int(pill_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=PILL
    obstacle_count=file.readline()
    for _ in range(int(obstacle_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=OBSTACLE
    initial_map["ploc"].append((0,0))
    return initial_map

 - <b>Actions:</b>

In [3]:
UP=(1,0)
DOWN=(-1,0)
RIGHT=(0,1)
LEFT=(0,-1)
ACTIONS=[UP,DOWN,LEFT,RIGHT]
ACT_DICT={UP:"U",DOWN:"D",RIGHT:"R",LEFT:"L"}

 - <b>Transition Model:</b>

In [4]:
def not_in_range(loc,loc_range):
    return loc[0]>=loc_range[0] or loc[1]>=loc_range[1] or loc[0]<0 or loc[1]<0
    

def move(current_state,player_number,direction):
    current_state=cp.deepcopy(current_state)
    current_location=current_state["ploc"][player_number]
    future_location=(current_location[0]+direction[0],current_location[1]+direction[1])
    if(future_location in current_state):
        if(current_state[future_location]==POTION):
            current_state["potion_count"]-=1;
        elif(current_state[future_location]==PILL):
            current_state["ploc"].append((current_state["row"]-1,0))
        elif(current_state[future_location]==OBSTACLE):
            return 0
    elif(not_in_range(future_location,(current_state["row"],current_state["col"]))):
        return 0
    current_location=current_state["ploc"][player_number]=future_location
    if(current_location in current_state):
        del current_state[current_location]
    return current_state

- <b>Goal State:</b>

In [5]:
def is_goal_state(state):
    for loc in state["ploc"]:
        if(loc!=(state["row"]-1,state["col"]-1)):
            return False
    if(state["potion_count"]==0):
        return True
    else:
        return False

# Breadth First Search (BFS)

## Algorithm
bellow the BFS algorithm is implemented. a dictionary is used to store explored states because it is faster to check the existance of a key in dictionary. Since a dictionary cannot be used as a key, we define a function "encode" that takes an state and turn it into hashable format and returns the hashed dictionary.
## Pros and Cons
pros:
 - Always returns the optimal solution
 
cons:
 - Requires more space than IDS (of order O($b^m$))
 - Visits more states and is slower than A*(depends on heuristic)

## Implementation

In [245]:
def encode(state):
    encoded=cp.deepcopy(state)
    encoded.pop("ploc",None)
    encoded.pop("order",None)
    encoded.pop("row",None)
    encoded.pop("col",None)
    for player in range(len(state["ploc"])):
        encoded[player]=state["ploc"][player]
    return hash(tuple(encoded.items()))

def BFS(file_address):
    total_visited_states=1;
    frontier_queue=[]
    visited_states={}
    frontier_queue.append(read_initial_state(file_address))
    visited_states[encode(frontier_queue[0])]=True
    frontier_queue[0]["order"]=""
    while(frontier_queue):
        current_state=frontier_queue.pop(0)
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                new_state=move(current_state,player,action)
                if(new_state==0):
                    continue
                total_visited_states+=1
                if((encode(new_state) in visited_states)):
                    continue
                #print(encode(new_state))
                visited_states[encode(new_state)]=True
                new_state["order"]=current_state["order"]+" "+str(player)+ACT_DICT[action]
                if(is_goal_state(new_state)):
                    return (new_state["order"],total_visited_states)
                frontier_queue.append(new_state) 
    return (0,0)

## Tests

In [247]:
tic=time.time()
BFS("Tests/test1.in")
toc=time.time()
time1=toc-tic
tic=time.time()
BFS("Tests/test1.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test1,visited_states1 =BFS("Tests/test1.in")
toc=time.time()
time3=toc-tic
average_time_BFS_test1=(time1+time2+time3)/3
print("Path=" ,result_test1, "\naverage time= ",average_time_BFS_test1,"\nvisited_states=",visited_states1)

Path=  0R 0R 0R 0U 0U 0U 1D 1U 1R 1R 1R 
average time=  0.1599898338317871 
visited_states= 3909


In [249]:
tic=time.time()
BFS("Tests/test2.in")
toc=time.time()
time1=toc-tic
tic=time.time()
BFS("Tests/test2.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test2,visited_states2=BFS("Tests/test2.in")
toc=time.time()
time3=toc-tic
average_time_BFS_test2=(time1+time2+time3)/3
print("Path=" ,result_test2, "\naverage time= ",average_time_BFS_test2,"\nvisited_states=",visited_states2)

Path=  0U 0U 0R 0R 0D 0U 0U 0R 
average time=  0.43068138758341473 
visited_states= 6711


In [250]:
tic=time.time()
BFS("Tests/test3.in")
toc=time.time()
time1=toc-tic
tic=time.time()
BFS("Tests/test3.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test3,visited_states3=BFS("Tests/test3.in")
toc=time.time()
time3=toc-tic
average_time_BFS_test3=(time1+time2+time3)/3
print("Path=" ,result_test3, "\naverage time= ",average_time_BFS_test3,"\nvisited_states=",visited_states3)

Path=  0U 0U 0R 0R 0D 0R 0R 0R 0U 0U 0U 0U 1R 1R 1R 1D 1U 1R 1R 
average time=  40.913156827290855 
visited_states= 557673


## Overall Results
|property|test1|test2|test3|
|:-:|:---:|:---:|:---:|
|**Time**          |160ms|431ms|40.913s|
|**States Visited**|3909|6711|557673|
|**Path Depth** |11|8|19|

# Iterative Deepening Search (IDS)

## Algorithm
bellow the IDS algorithm is implemented. "encode" function from BFS is used here as well. the IDS functions uses DFS as a subroutine, it runs DFA but specifies the max depth DFA should explore and starting from 1, increases the depth after each run untill goal state is reached  
## Pros and Cons
pros:
 - Always returns the optimal solution
 - Uses less space than A* and BFS
 
cons:
 - Visits more states and is slower than A*(depends on heuristic)

## Implementation

In [254]:
def DFS(initial_state,max_level):
    total_visited_states=1;
    frontier_queue=[]
    visited_states={}
    frontier_queue.append([0,initial_state])
    #visited_states[encode(frontier_queue[0])]=1
    initial_state["order"]=""
    while(frontier_queue):
        added_level,current_state=frontier_queue.pop()
        if((encode(current_state) in visited_states)):
            if(visited_states[encode(current_state)]<=added_level):
                continue
        visited_states[encode(current_state)]=added_level
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                new_state=move(current_state,player,action)
                if(new_state==0):
                    continue
                new_state["order"]=current_state["order"]+" "+str(player)+ACT_DICT[action]
                if(is_goal_state(new_state)):
                    return (new_state["order"],total_visited_states)
                total_visited_states+=1;
                if(added_level+1<max_level):
                    frontier_queue.append([added_level+1,new_state]) 
    return (0,total_visited_states)
def IDS(file_address):
    total_states=0
    level=1
    start_state=read_initial_state(file_address)
    start_state["order"]=""
    res,total=DFS(start_state,level)
    total_states+=total
    while res==0:
        #print(level)
        level+=1
        res,total=DFS(start_state,level)
        total_states+=total
    return (res,total_states)

In [256]:
tic=time.time()
IDS("Tests/test1.in")
toc=time.time()
time1=toc-tic
tic=time.time()
IDS("Tests/test1.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test1,visited_states1 =IDS("Tests/test1.in")
toc=time.time()
time3=toc-tic
average_time_IDS_test1=(time1+time2+time3)/3
print("Path=" ,result_test1, "\naverage time= ",average_time_IDS_test1,"\nvisited_states=",visited_states1)

Path=  0R 1D 1R 1R 1R 1U 0R 0R 0U 0U 0U 
average time=  0.9571897983551025 
visited_states= 22814


In [257]:
tic=time.time()
IDS("Tests/test2.in")
toc=time.time()
time1=toc-tic
tic=time.time()
IDS("Tests/test2.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test2,visited_states2 =IDS("Tests/test2.in")
toc=time.time()
time3=toc-tic
average_time_IDS_test2=(time1+time2+time3)/3
print("Path=" ,result_test2, "\naverage time= ",average_time_IDS_test2,"\nvisited_states=",visited_states2)

Path=  0U 0U 0R 0R 0D 0U 0R 0U 
average time=  1.1417025725046794 
visited_states= 25919


In [258]:
tic=time.time()
IDS("Tests/test3.in")
toc=time.time()
time1=toc-tic
tic=time.time()
IDS("Tests/test3.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test3,visited_states3=IDS("Tests/test3.in")
toc=time.time()
time3=toc-tic
average_time_IDS_test3=(time1+time2+time3)/3
print("Path=" ,result_test3, "\naverage time= ",average_time_IDS_test3,"\nvisited_states=",visited_states3)

Path=  0U 0U 0R 0R 1R 1R 1R 1D 1R 1R 1U 0D 0R 0R 0R 0U 0U 0U 0U 
average time=  139.5229388078054 
visited_states= 1809160


## Overall Results
|property|test1|test2|test3|
|:-:|:---:|:---:|:---:|
|**Time**          |950ms|1.142s|140.522s|
|**States Visited**|22814|25919|1809160|
|**Path Depth** |11|8|19|

# A*

## Algorithm
bellow the A* algorithm is implemented. the method used to keep the track of visited states is similar to previous parts. For this algorithm, a heap is used to extract the state with minimum cost at each step. the cost of a state is defined as follows:

$$f(n)=g(n)+h(n)$$

where $h(n)$ is the heuristic function that estimates the cost to reach the goal and $g(n)$ is the cost of reaching n from start state.

### heuristic function

for A* algorithm to be optimal, our heuristic function should be consistent. we define the following heuristic function and prove its consistency:
$$h(n)=\sum_{player}d_{Manhattan}(locatin(player),(row-1,col-1))$$

## Pros and Cons
pros:
 - Always returns the optimal solution
 - Uses less space than A* and BFS
 
cons:
 - Visits more states and is slower than A*(depends on heuristic)

In [263]:
def heuristic(state):
    player_steps=0
    min_row=state["row"]-1
    min_col=state["col"]-1
    col=min_col
    row=min_row
    for loc in state["ploc"]:
        player_steps=row-loc[0]+col-loc[1]
    #for key,val in state.items():
     #   if (type(key)==tuple):
     #       if(val==POTION):
     #           row=min_row-key[0]
     #           col=min_col-key[1]
     #           if col<0: col=0
     #           if row<0: row=0
     #           player_steps+=(col+row)
    return player_steps

In [264]:
def A_Star(file_address):
    seen_states=0;
    frontier_queue=[]
    visited_states={}
    initial_state=read_initial_state(file_address)
    hp.heappush(frontier_queue,(heuristic(initial_state),0,initial_state))
    visited_states[encode(initial_state)]=True
    initial_state["order"]=""
    initial_state["level"]=0
    while(frontier_queue):
        cost,order,current_state=hp.heappop(frontier_queue)
        visited_states[encode(current_state)]=True
        if(is_goal_state(current_state)):
            return (current_state["order"],seen_states)
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                seen_states+=1
                new_state=move(current_state,player,action)
                if(new_state==0):
                    continue
                if((encode(new_state) in visited_states)):
                    continue
                new_state["order"]=current_state["order"]+" "+str(player)+ACT_DICT[action]
                new_state["level"]=current_state["level"]+1
                hp.heappush(frontier_queue,(heuristic(new_state)+new_state["level"],seen_states,new_state)) 
    return 0

In [ ]:
tic=time.time()
A_Star("Tests/test1.in")
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test1.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test1,visited_states1 =A_Star("Tests/test1.in")
toc=time.time()
time3=toc-tic
average_time_A_Star_test1=(time1+time2+time3)/3
print("Path=" ,result_test1, "\naverage time= ",average_time_A_Star_test1,"\nvisited_states=",visited_states1)